In [2]:
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go


In [102]:
master_list = []
model_area = 'FSA'
model = 'FSA_Base_2020pop_V044.mdb'
output_folder = r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Allways_Latest_Master_Model_Simulations\Plots'
result_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Allways_Latest_Master_Model_Simulations\Model"
result_list = []
result_list.append(['2yr 24h No Climate Change','FSA_WWF_EX-2yr-24hr-AES_2020pop_Base.res1d',1])
result_list.append(['5yr 24h No Climate Change','FSA_WWF_EX-5yr-24hr-AES_2020pop_Base.res1d',1])
result_list.append(['10yr 24h No Climate Change','FSA_WWF_EX-10yr-24hr-AES_2020pop_Base.res1d',1])
# result_list.append(['Dry Weather Flow','FSA_DWF_2021-07-23_4d_2020pop_Base.res1d',0])
# result_list.append(['Jan 31 2020 Event','FSA_WWF_2020-01-29_6d_2020pop_Base.res1d',0])
# result_list.append(['Nov 15 2021 Event','FSA_WWF_2021-11-12_5d_2020pop_Base.res1d',0])

master_list.append([model_area,model,result_folder,output_folder,result_list])

In [ ]:
for m in master_list:
    model = m[0]
    output_folder = m[1]
    result_folder = m[2]
    output_folder = m[3]
    result_list = m[4]
    nodes_df = pd.read_csv(result_folder + '\\Connection_Stats_Nodes.csv',dtype={'MUID': str})
    lines_df =  pd.read_csv(result_folder + '\\Connection_Stats_Lines.csv',dtype={'MUID': str, 'From Node': str, 'To Node': str})
    for r in result_list:
        header = r[0]
        file = r[1]
        res1d = Res1D(result_folder + '\\' + file)
        r.append(res1d)
        for index, row in nodes_df[:1].iterrows():
            print(row['MUID'])
            for column in nodes_df.columns[1:]:
                print(column)

In [109]:
for m in master_list:
                
    for index, row in nodes_df[:1].iterrows():
        node = row['MUID']

        fig = go.Figure()
        
        time_bookends = [res1d.time_index[0],res1d.time_index[-1]]
        for column in nodes_df.columns[1:]:
            if not np.isnan(row[column]):
                fig.add_trace(go.Scatter(x=time_bookends, 
                                 y = [row[column],row[column]], 
                                 mode='lines',name=column))
        
        for i, r in enumerate(result_list): 

            if r[2] == 1:
                fig.add_trace(go.Scatter(x=r[3].time_index, 
                                         y = list(r[3].query.GetNodeValues(node, "WaterLevel")), 
                                         mode='lines',name=r[0]))

                    
        fig.update_layout(
            autosize=False,
            width = 1362,
            height=500,
            margin=dict(
                l=50,
                r=50,
                b=25,
                t=35,
                pad=4
                ),
            yaxis_title="HGL (m)" 
            )

        upstream_df = lines_df[lines_df.ToNode==node]
        upstream_lines = list(upstream_df.MUID)
        upstream_layers = list(upstream_df.Layer)
        upstreams = zip(upstream_lines,upstream_layers)

        downstream_df = lines_df[lines_df.FromNode==node]
        downstream_lines = list(downstream_df.MUID)
        downstream_layers = list(downstream_df.Layer)
        downstreams = zip(downstream_lines,downstream_layers)

        with open(output_folder + "\\" + model + "_WaterLevel_Node_" + node + ".html", 'w') as f:
            f.write("<h1>Water Level in node '" + node + "</h1>\n")
            f.write("<h2>Links to Connected Elements</h2>\n")
            for upstream in upstreams:
                path = output_folder + "\\" + model + "_Discharge_" + upstream[1] + "_" + upstream[0] + ".html"
                f.write('<a href="' + path + '">Upstream ' + upstream[1] + ' ' + upstream[0] + '</a>\n')
                f.write("<br>")
            for downstream in downstreams:
                path = output_folder + "\\" + model + "_Discharge_" + downstream[1] + "_" + downstream[0] + ".html"
                f.write('<a href="' + path + '">Downstream ' + downstream[1] + ' ' + downstream[0] + '</a>\n')
                f.write("<br>")
            f.write("<h2>" + r[0] + "</h2>\n")
            f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
        f.close()

#     for index, row in lines_df.iterrows():
#         pipe = row['MUID']
#         from_node = row['FromNode']
#         to_node = row['ToNode']
#         layer = row['Layer']
#         fig = go.Figure()
#         if layer == 'Link':
#             muid = pipe
#         else:
#             muid = layer + ':' + pipe
#         for i, r in enumerate(result_list): 

#             fig.add_trace(go.Scatter(x=res1d.time_index, 
#                                      y = list(res1d.query.GetReachEndValues(muid, "Discharge")), 
#                                      mode='lines',name='Discharge'))

#         fig.update_layout(
#             autosize=False,
#             width = 1362,
#             height=500,
#             margin=dict(
#                 l=50,
#                 r=50,
#                 b=25,
#                 t=35,
#                 pad=4
#                 ),
#             yaxis_title="Discharge (m<sup>3</sup>/s)" 
#             )

#         with open(output_folder + "\\" + model + "_Discharge_" + layer + "_" + pipe + ".html", 'w') as f:
#             f.write("<h1>Discharge in " + layer + " " + pipe + "</h1>\n")
#             f.write("<h2>Links to Connected Elements</h2>\n")

#             path = output_folder + "\\" + model + "_WaterLevel_Node_" + from_node + ".html"
#             f.write('<a href="' + path + '">Upstream Node ' + from_node + '</a>\n')
#             f.write("<br>")

#             try:
#                 path = output_folder + "\\" + model + "_WaterLevel_Node_" + to_node + ".html"
#                 f.write('<a href="' + path + '">Downstream Node ' + to_node + '</a>\n')
#             except:
#                 f.write('<p>There is no downstream node.</p>\n')
#             f.write("<br>")

#             f.write("<h2>" + r[0] + "</h2>\n")
#             f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
#         f.close()

        

In [106]:
r[2]
    

1

In [99]:
result_list

[['2yr 24h No Climate Change',
  'FSA_WWF_EX-2yr-24hr-AES_2020pop_Base.res1d',
  1,
  'Yep'],
 ['5yr 24h No Climate Change',
  'FSA_WWF_EX-5yr-24hr-AES_2020pop_Base.res1d',
  1,
  'Yep'],
 ['10yr 24h No Climate Change',
  'FSA_WWF_EX-10yr-24hr-AES_2020pop_Base.res1d',
  1,
  'Yep']]